In [2]:
import os
import glob

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import modules.gait_metrics as gm
import modules.pose_estimation as pe


load_dir = os.path.join('data', 'kinect', 'best pos')
save_dir = os.path.join('data', 'results')

save_name = 'kinect_gait_metrics.csv'

# All files with .pkl extension
file_paths = glob.glob(os.path.join(load_dir, '*.pkl'))
save_path = os.path.join(save_dir, save_name)

df_metrics = pd.read_csv(save_path, index_col=0)


kinect_id = '2014-12-16_P005_Post_004'

matching_files = [f for f in file_paths if kinect_id in f]

for file_path in matching_files:

    df_head_feet = pd.read_pickle(file_path)
    
    # Convert all position vectors to float type 
    # so they can be easily input to linear algebra functions
    df_head_feet = df_head_feet.applymap(pd.to_numeric)

    # %% Peak detection

    # Cluster frames with k means to locate the 4 walking passes
    frames = df_head_feet.index.values.reshape(-1, 1)
    k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

    foot_diff = df_head_feet.L_FOOT - df_head_feet.R_FOOT
    foot_dist = foot_diff.apply(np.linalg.norm)



In [11]:
frame_rate = 30

In [3]:
import modules.general as gen

frame_labels = k_means.labels_

pass_dfs = gm.split_by_pass(df_head_feet, frame_labels)

In [4]:
# Enforce consistent sides for the feet on all walking passes. 
# Also calculate the general direction of motion for each pass.
pass_dfs, pass_directions = zip(*[pe.consistent_sides(df) for df in pass_dfs])

In [5]:
# Estimate frames where foot contacts floor
pass_dfs_contact = [gm.foot_contacts(a, b) for a, b in zip(pass_dfs, pass_directions)]

In [9]:
df_contact_frames = pass_dfs_contact[0]

In [10]:
# Foot positions at the contact frames
df_contact_pos = gen.lookup_values(df_contact_frames, df_head_feet)

In [12]:
df_stride_length = df_contact_pos.diff(periods=-1).applymap(np.linalg.norm)

In [13]:
df_stride_time = abs(df_contact_frames.diff(periods=-1)) / 30

In [14]:
df_stride_velocity = df_stride_length / df_stride_time

In [15]:
series_contact = df_contact_frames.stack().sort_values()

In [16]:
df_contact = series_contact.reset_index()
df_contact.columns = ['Contact number', 'Part', 'Frame']

In [17]:
df_contact

,Contact number,Part,Frame
0,0,R_FOOT,234.0
1,0,L_FOOT,246.0
2,1,R_FOOT,262.0


In [18]:
df_contact_pos

,L_FOOT,R_FOOT
0,"[-18.4634, -63.777, 250.636]","[-79.2984, -59.659, 254.924]"
1,NaN,"[40.0823, -63.9547, 263.324]"
